<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231129-%E5%BE%97%E5%88%B0%E9%9A%8F%E6%9C%BA%E6%A0%B7%E6%9C%AC%E7%9A%84%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=phkMcpqNxAw637MtwZIPAEgnzL0epwXKQUPkHR-fLj0&tc=QorR5a37QLseRnzYGH0FITzfTOAMCYuFzUQpbn6g6LY&cc=EVwx7oknXVVeeIMp57QJIBFiNKsYuWE4gtg_PQSQKGQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmXMpMgmVV-BkjAtdRXieg36NraGudWTMlIpwA8Ujs6o_h-lWb1iBM

Successfully saved authorization token.


In [2]:
import os
import pandas as pd
import math

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/random_8000")

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-22').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1245


In [5]:
# 二阶谐波
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

harmonicIndependents = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2'])

def add_sincos(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

harmonicSentinel = dataset.map(add_sincos)

harmonicTrend = harmonicSentinel.select(harmonicIndependents.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents.length(),numY=1))

harmonicTrendCoefficients = harmonicTrend.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents])

def fit(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('fitted'))

fittedHarmonic = harmonicSentinel.map(fit)

print(fittedHarmonic.size().getInfo())

1245


In [6]:
# 5天序列NDVI
interval = 5
increment = 'day'
start = '2022-05-01'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-01').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  timeRadians2 = ee.Image(years).multiply(4 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDVI(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('NDVI'))

fitteds2 = s2makeup.map(add_fittedNDVI)

HANTS = fitteds2.select('NDVI')

In [7]:
# NDVI时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI']


In [8]:
image_agrgt_clip = image_agrgt.clip(roi)

In [9]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id'],scale=10,tileScale=8)

In [10]:
cols = ['id','0_NDVI','1_NDVI','2_NDVI','3_NDVI','4_NDVI','5_NDVI','6_NDVI','7_NDVI','8_NDVI','9_NDVI','10_NDVI','11_NDVI','12_NDVI','13_NDVI','14_NDVI','15_NDVI','16_NDVI','17_NDVI','18_NDVI','19_NDVI','20_NDVI','21_NDVI','22_NDVI','23_NDVI','24_NDVI']
inds = range(samples_datesets.size().getInfo())

In [11]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,0_NDVI,1_NDVI,2_NDVI,3_NDVI,4_NDVI,5_NDVI,6_NDVI,7_NDVI,8_NDVI,...,15_NDVI,16_NDVI,17_NDVI,18_NDVI,19_NDVI,20_NDVI,21_NDVI,22_NDVI,23_NDVI,24_NDVI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(26),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
id,7001.000000,7002.000000,7003.000000,7004.000000,7005.000000,7006.000000,7007.000000,7008.000000,7009.000000,7010.000000,...,7991.000000,7992.000000,7993.000000,7994.000000,7995.000000,7996.000000,7997.000000,7998.000000,7999.000000,8000.000000
0_NDVI,0.518816,0.757307,0.676236,0.136585,0.679376,0.798911,-0.000587,0.723471,0.275889,0.831940,...,0.112542,0.772259,0.786445,0.429951,0.276511,0.644297,0.621578,0.646761,0.727018,0.729354
1_NDVI,0.563442,0.763865,0.688204,0.138153,0.686962,0.825186,-0.000487,0.737986,0.277510,0.846886,...,0.120894,0.784087,0.809511,0.434172,0.281360,0.655272,0.629696,0.652222,0.742182,0.743166
2_NDVI,0.604891,0.769795,0.698424,0.139146,0.693791,0.848844,-0.000215,0.751878,0.279131,0.859195,...,0.129803,0.794211,0.830278,0.438543,0.286457,0.665132,0.635632,0.658434,0.754510,0.756580
3_NDVI,0.641988,0.775133,0.707135,0.139550,0.700172,0.869565,0.000197,0.764943,0.280786,0.868795,...,0.139173,0.802714,0.848563,0.443036,0.291887,0.673844,0.639516,0.665465,0.763927,0.769382


In [13]:
df.to_csv('randomsample_NDVI_2022_8000.csv',encoding='utf-8')